In [1]:
# def recursive_extract(term, results=None):
#     """Recursively extract all terms from a complex Prolog structure."""
#     if results is None:
#         results = []
    
#     # If this isn't a Term, we can't process it
#     if not isinstance(term, Term):
#         return results
    
#     # Check if this is a conjunction (,) or disjunction (;)
#     if term.functor == ',' or term.functor == ';':
#         # Recursively process each argument
#         for arg in term.args:
#             recursive_extract(arg, results)
#     # If this is a rule (:-), process both head and body
#     elif term.functor == ':-' and term.arity == 2:
#         # Add the head to results (usually not a langda, but might be useful)
#         recursive_extract(term.args[0], results)
#         # Process the body recursively
#         recursive_extract(term.args[1], results)
#     else:
#         # This is a regular predicate term
#         results.append(term)
    
#     return results

# import problog
# import re
# from typing import List
# from problog.logic import Term

# class Langda(object):
#     def __init__(self, model_string, caching=False, saving=False):
#         self.model_string = self.parse(model_string)

#     def parse(self, model_string):
#         langda_string, out_model_string = list(), list()
#         parser = problog.parser.PrologParser(problog.program.ExtendedPrologFactory())
#         langda_pattern = r'langda\("([^"\\]*(\\.[^"\\]*)*?)"\)'
        
#         for line in model_string.split('\n'):
#             if 'langda' in line:
#                 # matches = list(re.finditer(langda_pattern, line))
#                 modified_line = re.sub(langda_pattern, "\n{LANGDA}\n", line)
#                 out_model_string.append(modified_line)
#                 # print(matches)
#                 # print(re.finditer(langda_pattern, line))
#                 # if matches:
#                 #     modified_line = line
#                 #     for match in matches:
#                 #         modified_line = modified_line.replace(match.group(0), "\n{LANGDA}\n")

#                 #         # print("match:",match.group(1))
#                 #         out_model_string.append(modified_line)

#             else:
#                 out_model_string.append(line)
    
#         return '\n'.join(out_model_string)

# l = Langda(model_string)
# print(l.model_string)

In [ ]:
# model_string = """
# langda("hello; there").
# vols(X) :-
#     (langda("nothing"), langda("Is this, this, as ggod?! oh no- have fun"), where(X,Y));house(X).
# valid_drone_flight(X) :-
#     what(X);langda("there's a high building at 0.11"), vlos(X),
#     here(X).
# % Definition of a valid mission
# landscape(X) :- 
#     (langda("stop from there"), vlos(X)), addtion(X), langda("danger, stop!"), langda("what the hell?").
# final(X) :- langda("23 is a good number; stop here").
# """



In [11]:
text = """
Here's the code to replace the `{LANGDA}` placeholder, which adds the construction site avoidance rule:

```
    distance(X, (500,500)) >= 20,
    vlos(X), weight < 25, can_return(X);
    distance(X, (500,500)) >= 20,
    permits(X), can_return(X)
```
```
    distance(X, (500,500)) >= 20,
    vlos(X), weight < 25, can_return(X);
    distance(X, (500,500)) >= 20,
    permits(X), can_return(X)
```
```
    distance(X, (500,500)) >= 20,
    vlos(X), weight < 25, can_return(X);
    distance(X, (500,500)) >= 20,
    permits(X), can_return(X)
```
This code:
1. Ensures the drone maintains at least 20 units distance from the construction site at (500,500)
2. Combines with both possible valid flight conditions (VLOS with weight check or permits)
3. Maintains all other existing safety requirements
4. Uses the same structure as the rest of the rules with semicolon-separated alternatives



"""
import re

generated_codes = []
pattern = r"```(?:prolog|[a-z]*)?\n(.*?)```"
matches = re.findall(pattern, text, re.DOTALL)
for match in matches:
    generated_codes.append(match)
print(len(generated_codes))

3


In [12]:
pattern = r"\{\{LANGDA\}\}"
model_string = """
{{LANGDA}}
vols(X) :-
    {{LANGDA}}
    , langda("Is this, this, as ggod?! oh no- have fun"), where(X,Y));house(X).
valid_drone_flight(X) :-
    what(X);langda("there's a high building at 0.11"), vlos(X),
    here(X).
% Definition of a valid mission
landscape(X) :- 
    (langda("stop from there"), vlos(X)), addtion(X), langda("danger, stop!"), langda("what the hell?").
final(X) :- langda("23 is a good number; stop here").
"""

repl_iter = iter(generated_codes)
res = re.sub(r"\{\{LANGDA\}\}", lambda _: next(repl_iter, "{{LANGDA}}"), model_string)

In [7]:
import problog
from problog.logic import Term, Constant, Var
from problog.evaluator import Semiring
import torch
import numpy as np
import os
import sys

project_root = os.path.abspath("/Users/zhenzhili/MASTERTHESIS/#Expert_System_Design")
if project_root not in sys.path:
    sys.path.insert(0, project_root)
from src.deepprobcep.model import Model
from src.deepprobcep.network import Network
from src.deepprobcep.standard_networks import FC

from src.deepprobcep.gradient_semiring import SemiringGradient
from src.deepprobcep.vector_shape import VectorShape

# Define a simple test function for the network
def test_nn(network, *inputs):
    return torch.FloatTensor([0.3, 0.7])  # Example probabilities for binary classification

# Create a simple model
model_string = "nn(test_net,[X],Y,[a,b]) :: test_pred(X,Y).\nquery(X) :- test_pred(X,a)."
fc = FC(1, 2)  # Simple fully connected network with 1 input and 2 outputs
net = Network(fc, 'test_net', test_nn)
model = Model(model_string, [net])
# Create a query
query = Term('query', Constant(5))

# Get the compiled SDD and shape
sdd, shape = model.get_sdd(query)

# Examine the shape
print("Shape length:", shape.length)
print("Shape indices:", shape.indices)
print("Shape networks:", shape.networks)
print("Shape parameters:", shape.parameters)

# Create the semiring
semiring = SemiringGradient(model, shape)

# Print semiring methods
print("\nSemiring zero:", semiring.zero())
print("Semiring one:", semiring.one())

# Create a valid list term for testing
# This is how ProbLog represents lists internally: .(head, tail)
empty_list = Term('[]')
list_term = Term('.', Constant(5), empty_list)  # A list with just one element: [5]

# Now create a valid nn term with the proper list format
test_term = Term('nn', Constant('test_net'), list_term, Constant('a'))

try:
    prob_value = semiring.value(test_term)
    print("\nProbability value for test_term:", prob_value)
except Exception as e:
    print("\nError evaluating test_term:", e)
    
    # Let's try to debug what's happening
    from logic import term2list2
    try:
        print("Converting list term to Python list:", term2list2(list_term))
    except Exception as e:
        print("Error converting list term:", e)

# Test plus method with valid values
a = semiring.zero()
b = semiring.one()
plus_result = semiring.plus(a, b)
print("\nPlus result:", plus_result)

# Test times method
times_result = semiring.times(a, b)
print("\nTimes result:", times_result)

# Let's also try examining the SDD structure
print("\nSDD structure:")
print(sdd)

# Execute the query to see what happens
try:
    result = model.solve(query)
    print("\nQuery result:", result)
    if query in result:
        print("\nGradients:", result[query][1])
except Exception as e:
    print("\nError solving query:", e)

Shape length: 2
Shape indices: [0]
Shape networks: [('test_net', [5])]
Shape parameters: []

Semiring zero: (0.0, array([0., 0.]))
Semiring one: (1.0, array([0., 0.]))

Probability value for test_term: (0.30000001192092896, array([1., 0.]))

Plus result: (1.0, array([0., 0.]))

Times result: (0.0, array([0., 0.]))

SDD structure:
1: atom(identifier=(12, (5,) {{}}, 0), probability=nn(test_net,[5],a), group=(12, (5,) {{}}), name=query(5), source=None, is_extra=False)
Queries : 
* query(5) : 1 [query]


Query result: {query(5): (0.30000001192092896, {('test_net', [5]): array([1., 0.])})}

Gradients: {('test_net', [5]): array([1., 0.])}


In [74]:
text = '''
% =============================== % EXAMPLE % =============================== %
% UAV properties
initial_charge ~ normal(90, 5).
charge_cost ~ normal(-0.1, 0.2).
weight ~ normal(0.2, 0.1).

% Weather conditions
1/10::fog; 9/10::clear.

% Visual line of sight
vlos(X) :- 
    fog, distance(X, operator) < 50;
    clear, distance(X, operator) < 100;
    clear, over(X, bay), distance(X, operator) < 400.

langda(X:"answer",Desc:"nothing").
langda(X:"answer",Net:"[mlp(1,2,3,4,5,6)]").

% Sufficient charge to return to operator
can_return(X) :- 
    B is initial_charge, O is charge_cost,
    D is distance(X, operator), 0 < B + (2 * O * D).

% Permits related to local features
permits(X) :- 
    distance(X, service) < 15; distance(X, primary) < 15;
    distance(X, secondary) < 10; distance(X, tertiary) < 5;
    distance(X, crossing) < 5; distance(X, rail) < 5;
    over(X, park).

valid_drone_flight(X) :-
    (langda(X:"Return",T:"Time",Desc"there's a high building at (100,200)",Net:"[mnist_net1(0,1), mnist_net2(2,3)]",), vlos(X));(can_return(X));langda(T:"anstimewer",Net:"[mlp(4,5,6)]").

coin_win() :-
    langda(X:"return", T:"time", Net:"[mnist_net1(0,1), mnist_net2(2,3)]", 
    Desc:"If the code difference is 2, it returns 1, otherwise it returns 0."), 
    permits(X), langda(X:"return", Y:"fun", Z:"darmstadt", Net:"[position_net(there,here), audio(dog,child,horn)]", Desc:"If the we is 2, and horn is there, we eat banana.")
    
% Definition of a valid mission
landscape(X) :- 
    vlos(X), weight < 25, can_return(X); 
    permits(X), can_return(X).
'''

In [32]:
import re

pattern = r'langda\s*\(\s*((?:\s*\w+\s*:\s*(?:"[^"]*"|\[[^\]]*\]|\S+)\s*,?)+)\s*\).'

match = re.findall(pattern, text, re.S)
if match:
    # group(1) 捕获的是内部的键值对部分
    print("匹配到的内容:")
    print(match)
else:
    print("未匹配到")

匹配到的内容:
['X:"answer",Desc:"nothing"', 'X:"return", T:"time", Net:[mnist_net1(0,1), mnist_net2(2,3)], \n    Desc:"If the code difference is 2, it returns 1, otherwise it returns 0."', 'X:"return", Y:"fun", Z:"darmstadt", Net:[position_net("there","here"), audio("dog","child","horn")], Desc:"If the we is 2, and horn is there, we eat banana."']


In [ ]:
requirements_list, out_rule_string = list(), list()
pattern = r'langda\s*\(\s*((?:\s*\w+\s*:\s*(?:"[^"]*"|\[[^\]]*\]|\S+)\s*,?)+)\s*\).' # for langda(X:"str",Y:"str",...). format

for line in text.split('\n'):
    if 'langda' in line:
        modified_line = re.sub(
            pattern, 
            lambda m: (requirements_list.append(f"\nRequirement:\n{m.group(1)}\n")) or "\n{{LANGDA}}\n", 
            line,
            flags=re.DOTALL
        )
        out_rule_string.append(modified_line)
    else:
        out_rule_string.append(line)


print('\n'.join(out_rule_string))
print(out_rule_string)



替换后的文本：

% =============================== % EXAMPLE % =============================== %
% UAV properties
initial_charge ~ normal(90, 5).
charge_cost ~ normal(-0.1, 0.2).
weight ~ normal(0.2, 0.1).

% Weather conditions
1/10::fog; 9/10::clear.

% Visual line of sight
vlos(X) :- 
    fog, distance(X, operator) < 50;
    clear, distance(X, operator) < 100;
    clear, over(X, bay), distance(X, operator) < 400.

{LANGDA}.
{LANGDA}.

% Sufficient charge to return to operator
can_return(X) :- 
    B is initial_charge, O is charge_cost,
    D is distance(X, operator), 0 < B + (2 * O * D).

% Permits related to local features
permits(X) :- 
    distance(X, service) < 15; distance(X, primary) < 15;
    distance(X, secondary) < 10; distance(X, tertiary) < 5;
    distance(X, crossing) < 5; distance(X, rail) < 5;
    over(X, park).

valid_drone_flight(X) :-
    langda(X:"Return",T:"Time",Desc"there's a high building at (100,200)",Net:"[mnist_net1(0,1), mnist_net2(2,3)]",), vlos(X).

coin_win() :-


In [45]:
import re

def process_text_with_langda(text):
    """
    处理包含langda表达式的文本，包括跨多行的langda表达式
    
    Args:
        text (str): 原始文本
        
    Returns:
        list: 处理后的文本行列表
        list: 提取的requirements列表
    """
    # 先把整个文本合并成一个大字符串，以便处理跨行的langda
    full_text = text
    
    # 创建空列表存储结果
    requirements_list = []
    
    # 查找所有langda表达式
    langda_positions = []
    bracket_stack = []
    in_langda = False
    in_quotes = False
    is_escaped = False
    langda_start = -1
    
    # 第一遍扫描：找出所有langda表达式的位置
    i = 0
    while i < len(full_text):
        # 处理引号和转义字符
        if in_langda:
            if full_text[i] == '"' and not is_escaped:
                in_quotes = not in_quotes
            
            is_escaped = full_text[i] == '\\' and not is_escaped
            
            # 只有在不在引号中时才处理括号
            if not in_quotes:
                if full_text[i] == '(':
                    bracket_stack.append('(')
                elif full_text[i] == ')':
                    if bracket_stack:  # 确保栈不为空
                        bracket_stack.pop()
                        # 当括号栈为空时，标志着langda表达式结束
                        if not bracket_stack:
                            langda_positions.append((langda_start, i + 1))
                            in_langda = False
            
        # 检查是否遇到langda表达式的开始
        elif full_text[i:i+6] == 'langda' and i+6 < len(full_text) and full_text[i+6] == '(':
            in_langda = True
            langda_start = i
            bracket_stack.append('(')
            i += 7  # 跳过'langda('
            continue
        
        i += 1
    
    # 处理找到的langda表达式
    result_text = full_text
    offset = 0  # 位置偏移量
    
    for start, end in langda_positions:
        # 提取langda表达式
        langda_expr = full_text[start:end]
        
        # 从langda表达式中提取参数部分
        params_match = re.search(r'langda\s*\(\s*([\s\S]*)\s*\)$', langda_expr)
        if params_match:
            params = params_match.group(1)
            requirements_list.append(f"\nRequirement:\n{params}\n")
            
            # 替换原文本中的langda表达式
            adjusted_start = start + offset
            adjusted_end = end + offset
            result_text = result_text[:adjusted_start] + "{{LANGDA}}" + result_text[adjusted_end:]
            
            # 更新偏移量
            offset += len("{{LANGDA}}") - (end - start)
    
    # 将结果文本分割成行
    out_rule_string = result_text.split('\n')
    
    return out_rule_string, requirements_list



if __name__ == "__main__":
    # 处理文本
    out_rule_string, requirements_list = process_text_with_langda(text)
    
    # 打印结果
    print("处理后的文本:")
    print('\n'.join(out_rule_string))
    print("\n提取的requirements:")
    for req in requirements_list:
        print(req)

处理后的文本:

% =============================== % EXAMPLE % =============================== %
% UAV properties
initial_charge ~ normal(90, 5).
charge_cost ~ normal(-0.1, 0.2).
weight ~ normal(0.2, 0.1).

% Weather conditions
1/10::fog; 9/10::clear.

% Visual line of sight
vlos(X) :- 
    fog, distance(X, operator) < 50;
    clear, distance(X, operator) < 100;
    clear, over(X, bay), distance(X, operator) < 400.

{{LANGDA}}.
{{LANGDA}}.

% Sufficient charge to return to operator
can_return(X) :- 
    B is initial_charge, O is charge_cost,
    D is distance(X, operator), 0 < B + (2 * O * D).

% Permits related to local features
permits(X) :- 
    distance(X, service) < 15; distance(X, primary) < 15;
    distance(X, secondary) < 10; distance(X, tertiary) < 5;
    distance(X, crossing) < 5; distance(X, rail) < 5;
    over(X, park).

valid_drone_flight(X) :-
    ({{LANGDA}}, vlos(X));(can_return(X));{{LANGDA}}.

coin_win() :-
    {{LANGDA}}, 
    permits(X), {{LANGDA}}

% Definition of a valid 

# v2:

In [ ]:
import re

def process_text_with_langda(text):
    """
    处理包含langda表达式的文本，包括跨多行的langda表达式
    
    Args:
        text (str): 原始文本
        
    Returns:
        list: 处理后的文本行列表
        list: 提取的requirements列表
    """
    full_text = text          # 先把整个文本合并成一个大字符串，以便处理跨行的langda
    requirements_list = []    # 创建空列表存储结果
    langda_positions = []     # 查找所有langda表达式
    bracket_stack = []
    in_langda, in_quotes, is_escaped = False, False, False
    langda_start = -1
    
    i = 0    # 第一遍扫描：找出所有langda表达式的位置
    while i < len(full_text):
        # 处理引号和转义字符
        if in_langda:
            if full_text[i] == '"' and not is_escaped:
                in_quotes = not in_quotes
            
            is_escaped = full_text[i] == '\\' and not is_escaped
            
            # 只有在不在引号中时才处理括号
            if not in_quotes:
                if full_text[i] == '(':
                    bracket_stack.append('(')
                elif full_text[i] == ')':
                    if bracket_stack:  # 确保栈不为空
                        bracket_stack.pop()
                        # 当括号栈为空时，标志着langda表达式结束
                        if not bracket_stack:
                            langda_positions.append((langda_start, i + 1))
                            in_langda = False
            
        # 检查是否遇到langda表达式的开始
        elif full_text[i:i+6] == 'langda' and i+6 < len(full_text) and full_text[i+6] == '(':
            in_langda = True
            langda_start = i
            bracket_stack.append('(')
            i += 7  # 跳过'langda('
            continue
        
        i += 1
    
    # 处理找到的langda表达式
    result_text = full_text
    offset = 0  # 位置偏移量
    
    for start, end in langda_positions:
        # 提取langda表达式
        langda_expr = full_text[start:end]
        
        # 从langda表达式中提取参数部分
        params_match = re.search(r'langda\s*\(\s*([\s\S]*)\s*\)$', langda_expr)
        if params_match:
            params = params_match.group(1)
            requirements_list.append(f"\nRequirement:\n{params}\n")
            
            # 替换原文本中的langda表达式
            adjusted_start = start + offset
            adjusted_end = end + offset
            result_text = result_text[:adjusted_start] + "{{LANGDA}}" + result_text[adjusted_end:]
            
            # 更新偏移量
            offset += len("{{LANGDA}}") - (end - start)
    
    # 将结果文本分割成行
    out_rule_string = result_text.split('\n')
    
    return out_rule_string, requirements_list


# 测试函数
def test_parser():
    # 测试文本

    # 处理文本
    out_rule_string, requirements_list = process_text_with_langda(text)
    
    # 打印结果
    print("处理后的文本:")
    print('\n'.join(out_rule_string))
    print("\n提取的requirements:")
    for req in requirements_list:
        print(req)


if __name__ == "__main__":
    test_parser()

处理后的文本:

% =============================== % EXAMPLE % =============================== %
% UAV properties
initial_charge ~ normal(90, 5).
charge_cost ~ normal(-0.1, 0.2).
weight ~ normal(0.2, 0.1).

% Weather conditions
1/10::fog; 9/10::clear.

% Visual line of sight
vlos(X) :- 
    fog, distance(X, operator) < 50;
    clear, distance(X, operator) < 100;
    clear, over(X, bay), distance(X, operator) < 400.

{{LANGDA}}.
{{LANGDA}}.

% Sufficient charge to return to operator
can_return(X) :- 
    B is initial_charge, O is charge_cost,
    D is distance(X, operator), 0 < B + (2 * O * D).

% Permits related to local features
permits(X) :- 
    distance(X, service) < 15; distance(X, primary) < 15;
    distance(X, secondary) < 10; distance(X, tertiary) < 5;
    distance(X, crossing) < 5; distance(X, rail) < 5;
    over(X, park).

valid_drone_flight(X) :-
    ({{LANGDA}}, vlos(X));(can_return(X));{{LANGDA}}.

coin_win() :-
    {{LANGDA}}, 
    permits(X), {{LANGDA}}

% Definition of a valid 

# v3: error exception:

In [81]:
import re

def process_text_with_langda(text):
    """
    处理包含langda表达式的文本，使用"先替换后验证"的策略
    如有语法错误，会在运行时抛出相应的异常
    
    Args:
        text (str): 原始文本
        
    Returns:
        list: 处理后的文本行列表
        list: 提取的requirements列表
    """
    # 保存原始行与处理后行的映射，用于错误定位
    original_lines = text.split('\n')
    line_map = {}
    
    # 创建空列表存储结果
    requirements_list = []
    processed_lines = []
    
    # 使用状态变量跟踪当前处理的langda表达式
    current_langda = ""
    in_langda = False
    line_start = -1
    bracket_count = 0
    
    # 逐行处理文本
    for i, line in enumerate(original_lines):
        # 如果当前不在langda表达式内，查找新的langda开始
        if not in_langda:
            langda_start = line.find("langda(")
            if langda_start != -1:
                # 找到新的langda表达式开始
                in_langda = True
                line_start = i
                current_langda = line[langda_start:]
                # 计算初始括号数量
                bracket_count = current_langda.count('(') - current_langda.count(')')
                
                # 如果当前行就完成了langda表达式
                if bracket_count == 0:
                    # 提取表达式参数
                    try:
                        params = re.search(r'langda\s*\(\s*(.*)\s*\)', current_langda).group(1)
                        requirements_list.append(f"\nRequirement:\n{params}\n")
                    except (AttributeError, IndexError):
                        # 如果提取失败，仍然保留错误，让运行时检测
                        pass
                    
                    # 替换langda表达式
                    processed_line = line.replace(current_langda, "{{LANGDA}}")
                    processed_lines.append(processed_line)
                    line_map[len(processed_lines) - 1] = i
                    
                    # 重置状态
                    in_langda = False
                    current_langda = ""
                else:
                    # 当前行没有完成langda表达式，保存当前行，等待完成后再处理
                    processed_lines.append(line)
                    line_map[len(processed_lines) - 1] = i
            else:
                # 当前行没有langda表达式
                processed_lines.append(line)
                line_map[len(processed_lines) - 1] = i
        else:
            # 当前在langda表达式内，继续累积表达式内容
            current_langda += "\n" + line
            bracket_count += line.count('(') - line.count(')')
            
            # 检查是否完成了当前langda表达式
            if bracket_count == 0:
                # 提取表达式参数
                try:
                    params = re.search(r'langda\s*\(\s*(.*)\s*\)', current_langda).group(1)
                    requirements_list.append(f"\nRequirement:\n{params}\n")
                except (AttributeError, IndexError):
                    # 如果提取失败，仍然保留错误，让运行时检测
                    pass
                
                # 替换从开始行到当前行的所有内容
                start_line = processed_lines[line_start]
                langda_start = start_line.find("langda(")
                
                # 替换开始行
                new_start_line = start_line[:langda_start] + "{{LANGDA}}"
                processed_lines[line_start] = new_start_line
                
                # 删除中间行 (如果有)
                while len(processed_lines) > line_start + 1 and len(processed_lines) - 1 < i:
                    processed_lines.pop(line_start + 1)
                
                # 添加当前行的剩余部分(如果有)
                if i > line_start:
                    end_of_langda = line.find(')')
                    if end_of_langda != -1 and end_of_langda + 1 < len(line):
                        processed_lines.append(line[end_of_langda + 1:])
                        line_map[len(processed_lines) - 1] = i
                
                # 重置状态
                in_langda = False
                current_langda = ""
            else:
                # 当前langda表达式未完成，不添加当前行
                # 因为之前已经添加了起始行，后面会统一处理
                pass
    
    # 检查是否有未完成的langda表达式
    if in_langda:
        # 让解释器在运行时检测错误，不在这里处理
        pass
    
    return processed_lines, requirements_list, line_map


# 测试函数 - 用于演示如何使用解析器并处理可能的错误
def test_parser():

    
    try:
            
        # 测试运行时错误检测
        try:
            # 假设这里执行处理后的代码，如果有语法错误会抛出异常
            out_rule_string, requirements_list, line_map = process_text_with_langda(text)
            print("处理后的文本:")
            print('\n'.join(out_rule_string))
            
            # 模拟运行时错误
            exec('\n'.join(out_rule_string))
        except SyntaxError as e:
            # 捕获到语法错误
            print(f"捕获到运行时语法错误: {e}")
            # 使用行映射定位到原始文本中的错误位置
            if hasattr(e, 'lineno') and e.lineno in line_map:
                original_line = line_map[e.lineno]
                print(f"错误发生在原始文本的第{original_line + 1}行")
    except Exception as e:
        print(f"处理时发生错误: {e}")


if __name__ == "__main__":
    test_parser()

处理后的文本:

% =============================== % EXAMPLE % =============================== %
% UAV properties
initial_charge ~ normal(90, 5).
charge_cost ~ normal(-0.1, 0.2).
weight ~ normal(0.2, 0.1).

% Weather conditions
1/10::fog; 9/10::clear.

% Visual line of sight
vlos(X) :- 
    fog, distance(X, operator) < 50;
    clear, distance(X, operator) < 100;
    clear, over(X, bay), distance(X, operator) < 400.

{{LANGDA}}
{{LANGDA}}

% Sufficient charge to return to operator
can_return(X) :- 
    B is initial_charge, O is charge_cost,
    D is distance(X, operator), 0 < B + (2 * O * D).

% Permits related to local features
permits(X) :- 
    distance(X, service) < 15; distance(X, primary) < 15;
    distance(X, secondary) < 10; distance(X, tertiary) < 5;
    distance(X, crossing) < 5; distance(X, rail) < 5;
    over(X, park).

valid_drone_flight(X) :-
    ({{LANGDA}}
, mnist_net2(2,3)]", 
    Desc:"If the code difference is 2, it returns 1, otherwise it returns 0."), 
    permits(X), {{LAN

# V4: replace langda(X,Y,T)

In [73]:
import re

def process_text_with_langda(text):
    """
    处理包含langda表达式的文本，使用"先替换后验证"的策略
    如有语法错误，会在运行时抛出相应的异常
    
    Args:
        text (str): 原始文本
        
    Returns:
        list: 处理后的文本行列表
        list: 提取的requirements列表
    """
    # 保存原始行与处理后行的映射，用于错误定位
    original_lines = text.split('\n')
    line_map = {}
    
    # 创建空列表存储结果
    requirements_list = []
    processed_lines = []
    
    # 使用状态变量跟踪当前处理的langda表达式
    current_langda = ""
    in_langda = False
    line_start = -1
    bracket_count = 0
    
    # 逐行处理文本
    for i, line in enumerate(original_lines):
        # 如果当前不在langda表达式内，查找新的langda开始
        if not in_langda:
            langda_start = line.find("langda(")
            if langda_start != -1:
                # 找到新的langda表达式开始
                in_langda = True
                line_start = i
                current_langda = line[langda_start:]
                # 计算初始括号数量
                bracket_count = current_langda.count('(') - current_langda.count(')')
                
                # 如果当前行就完成了langda表达式
                if bracket_count == 0:
                    # 提取表达式参数
                    try:
                        params = re.search(r'langda\s*\(\s*(.*)\s*\)', current_langda).group(1)
                        requirements_list.append(f"\nRequirement:\n{params}\n")
                    except (AttributeError, IndexError):
                        # 如果提取失败，仍然保留错误，让运行时检测
                        pass
                    
                    # 提取参数中的键值对，除了Net和Desc之外的所有参数
                    key_values = re.findall(r'(\w+)\s*:\s*"([^"]*)"', params)
                    param_keys = [k for k, v in key_values if k not in ["Net", "Desc"]]
                    
                    # 生成替换内容：langda(X,Y,Z)，其中X,Y,Z是原始langda中的参数名
                    replacement = f"langda({','.join(param_keys)})" if param_keys else "langda()"
                    
                    # 替换langda表达式
                    processed_line = line.replace(current_langda, replacement)
                    processed_lines.append(processed_line)
                    line_map[len(processed_lines) - 1] = i
                    
                    # 重置状态
                    in_langda = False
                    current_langda = ""
                else:
                    # 当前行没有完成langda表达式，保存当前行，等待完成后再处理
                    processed_lines.append(line)
                    line_map[len(processed_lines) - 1] = i
            else:
                # 当前行没有langda表达式
                processed_lines.append(line)
                line_map[len(processed_lines) - 1] = i
        else:
            # 当前在langda表达式内，继续累积表达式内容
            current_langda += "\n" + line
            bracket_count += line.count('(') - line.count(')')
            
            # 检查是否完成了当前langda表达式
            if bracket_count == 0:
                # 提取表达式参数
                try:
                    params = re.search(r'langda\s*\(\s*(.*)\s*\)', current_langda).group(1)
                    requirements_list.append(f"\nRequirement:\n{params}\n")
                except (AttributeError, IndexError):
                    # 如果提取失败，仍然保留错误，让运行时检测
                    pass
                
                # 替换从开始行到当前行的所有内容
                start_line = processed_lines[line_start]
                langda_start = start_line.find("langda(")
                
                # 提取参数中的键值对，除了Net和Desc之外的所有参数
                key_values = re.findall(r'(\w+)\s*:\s*"([^"]*)"', params)
                param_keys = [k for k, v in key_values if k not in ["Net", "Desc"]]
                
                # 生成替换内容：langda(X,Y,Z)，其中X,Y,Z是原始langda中的参数名
                replacement = f"langda({','.join(param_keys)})" if param_keys else "langda()"
                
                # 替换开始行
                new_start_line = start_line[:langda_start] + replacement
                processed_lines[line_start] = new_start_line
                
                # 删除中间行 (如果有)
                while len(processed_lines) > line_start + 1 and len(processed_lines) - 1 < i:
                    processed_lines.pop(line_start + 1)
                
                # 添加当前行的剩余部分(如果有)
                if i > line_start:
                    end_of_langda = line.find(')')
                    if end_of_langda != -1 and end_of_langda + 1 < len(line):
                        processed_lines.append(line[end_of_langda + 1:])
                        line_map[len(processed_lines) - 1] = i
                
                # 重置状态
                in_langda = False
                current_langda = ""
            else:
                # 当前langda表达式未完成，不添加当前行
                # 因为之前已经添加了起始行，后面会统一处理
                pass
    
    # 检查是否有未完成的langda表达式
    if in_langda:
        # 让解释器在运行时检测错误，不在这里处理
        pass
    
    return processed_lines, requirements_list, line_map


# 测试函数 - 用于演示如何使用解析器并处理可能的错误
def test_parser():
    # 测试正确的文本

    try:
            
        # 测试运行时错误检测
        try:
            # 假设这里执行处理后的代码，如果有语法错误会抛出异常
            print("\n=== 测试缺少右括号的文本 ===")
            out_rule_string, requirements_list, line_map = process_text_with_langda(text)
            print("处理后的文本:")
            print('\n'.join(out_rule_string))
            
            # 模拟运行时错误
            exec('\n'.join(out_rule_string))
        except SyntaxError as e:
            # 捕获到语法错误
            print(f"捕获到运行时语法错误: {e}")
            # 使用行映射定位到原始文本中的错误位置
            if hasattr(e, 'lineno') and e.lineno in line_map:
                original_line = line_map[e.lineno]
                print(f"错误发生在原始文本的第{original_line + 1}行")
    except Exception as e:
        print(f"处理时发生错误: {e}")


if __name__ == "__main__":
    test_parser()


=== 测试缺少右括号的文本 ===
处理后的文本:

% =============================== % EXAMPLE % =============================== %
% UAV properties
initial_charge ~ normal(90, 5).
charge_cost ~ normal(-0.1, 0.2).
weight ~ normal(0.2, 0.1).

% Weather conditions
1/10::fog; 9/10::clear.

% Visual line of sight
vlos(X) :- 
    fog, distance(X, operator) < 50;
    clear, distance(X, operator) < 100;
    clear, over(X, bay), distance(X, operator) < 400.

langda(X)
langda(X)

% Sufficient charge to return to operator
can_return(X) :- 
    B is initial_charge, O is charge_cost,
    D is distance(X, operator), 0 < B + (2 * O * D).

% Permits related to local features
permits(X) :- 
    distance(X, service) < 15; distance(X, primary) < 15;
    distance(X, secondary) < 10; distance(X, tertiary) < 5;
    distance(X, crossing) < 5; distance(X, rail) < 5;
    over(X, park).

valid_drone_flight(X) :-
    (langda(X,T,T)
, mnist_net2(2,3)]", 
    Desc:"If the code difference is 2, it returns 1, otherwise it returns 0."), 


In [ ]:
# process each line:
original_lines = text.split('\n')

for i, line in enumerate(original_lines):
    # 如果当前不在langda表达式内，查找新的langda开始
    if not False:
        langda_start = line.find("langda(")
        


1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
